In [ ]:
import numpy as np
import cv2
import pandas as pd
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope


In [ ]:
import cv2
import numpy as np
import keras

# Load model
model = keras.models.load_model('/content/drive/MyDrive/Models/model_lane_dplv3_97.26%.h5')

# Load video
video_path = '/content/LD p2.mp4'
cap = cv2.VideoCapture(video_path)

# Define color for lane overlay
color = (0, 0, 255)  

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

# Define output video codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('/content/LD_op2_red.mp4', fourcc, fps, frame_size)

while True:
    # Read frame
    ret, frame = cap.read()

    if not ret:
        break

    # Resize frame to (224, 224)
    frame_resized = cv2.resize(frame, (224, 224))

    # Preprocess frame
    frame_input = np.expand_dims(frame_resized, axis=0)

    # Make predictions
    predictions = model.predict(frame_input)

    # Convert predictions to binary mask
    mask = (predictions > 0.5).astype(np.uint8)

    # Create a yellow mask
    lower_yellow = np.array([144, 238, 144])
    upper_yellow = np.array([0,100,0])
    frame_RGB = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
    mask_yellow = cv2.inRange(frame_RGB, lower_yellow, upper_yellow)

    # Resize yellow mask to match shape of mask
    mask_yellow_resized = np.reshape(mask_yellow, (1, 224, 224, 1)).astype(np.uint8)

    # Combine lane mask and yellow mask
    mask_combined = cv2.bitwise_or(mask, mask_yellow_resized)
    mask_combined = np.squeeze(mask_combined, axis=0)  # Remove extra dimension
    mask_combined = mask_combined.astype(np.uint8)

    # Apply mask to frame
    img_lane = cv2.bitwise_and(frame_resized, frame_resized, mask=mask_combined)

    # Resize lane image to original size
    img_lane_resized = cv2.resize(img_lane, (frame.shape[1], frame.shape[0]))

    # Overlay lane image on original frame
    img_lane_overlay = cv2.addWeighted(frame, 1, img_lane_resized, 0.5, 0)
    # Change the color of the lane overlay from white to red
    # img_lane_overlay[np.where((img_lane_overlay == [255, 255, 255]).all(axis=2))] = color
    # Write frame to output video
    out.write(img_lane_overlay)

# Release video capture and close windows
cap.release()
out.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 27ms/step


In [ ]:
import cv2
import numpy as np
import keras

# Load model
model = keras.models.load_model('/content/drive/MyDrive/Models/model_lane_dplv3_97.26%.h5')

# Load video
video_path = '/content/drive/MyDrive/LD p1.mp4'
cap = cv2.VideoCapture(video_path)

# Define color for lane overlay
color = (0, 0, 255)

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

# Define output video codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('/content/LD_op2_red.mp4', fourcc, fps, frame_size)

while True:
    # Read frame
    ret, frame = cap.read()

    if not ret:
        break

    # Resize frame to (224, 224)
    frame_resized = cv2.resize(frame, (224, 224))

    # Preprocess frame
    frame_input = np.expand_dims(frame_resized, axis=0)

    # Make predictions
    predictions = model.predict(frame_input)

    # Convert predictions to binary mask
    mask = (predictions > 0.5).astype(np.uint8)

    # Resize mask to match shape of frame
    mask_resized = cv2.resize(mask[0], (frame.shape[1], frame.shape[0]))

    # Apply color overlay to lane pixels
    img_lane_overlay = cv2.addWeighted(frame, 1, np.zeros_like(frame), 0, 0)
    img_lane_overlay[np.where(mask_resized == 1)] = color

    # Write frame to output video
    out.write(img_lane_overlay)

# Release video capture and close windows
cap.release()
out.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 27ms/step


In [ ]:
!pip install scikit-image

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install imageio-ffmpeg


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 45.7 MB/s eta 0:00:00
